In [2]:
import numpy as np

from my_stuff import MufexKeys
from quantfreedom.custom_logger import CustomLogger
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb
logger = CustomLogger(log_debug=True,disable_logging=False)

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 03 seconds to download the candles


In [4]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([1.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.Provided]),
    max_equity_risk_pct=np.array([4]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([150, 250]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [8]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles.values,
    logger=logger,
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
    logger=logger,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 2
Total combinations of settings to test: 2

Total candles: 2,000
Total candles to test: 4,000


In [9]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
1,0,0,97.0,-109.977185,27.84,-0.872501,-523.7581,476.242216
0,0,1,92.0,-110.827194,25.00,-0.864312,-525.6781,474.322113


In [14]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.RealBacktest,
    candles=candles.values,
    logger=logger,
)
candle_backtest = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
    logger=logger,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 2
Total combinations of settings to test: 2

Total candles: 2,000
Total candles to test: 4,000


In [15]:
candle_backtest.sort_values(['ind_set_idx','or_set_idx'])

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,0,0,69.0,-114.622020,17.39,-0.905231,-534.0647,465.935415
1,0,1,69.0,-147.508619,14.49,-0.917790,-595.9736,404.026334
